# Overlay Satellite Imagery with Contours

In order to successfully make a plot with satellite data and contours we'll need to use two data sources to bring in the data and then set the plot parameters for each. Some key elements to keep in mind:
1. The time of your satellite file likely won't be the same as the contour data.
   - This is because we get updated satellite data every 5 minutes while model analysis files are only available at 00, 06, 12, and 18 UTC.
   - Create two different time variables to help keep track of the different times.
2. You'll need to keep two separate variables for the data objects, one for the satellite data and one for the gridded data.
   - It doesn't matter what you call the variable names, for example you can store the satellite data in `ds` and the gridded data in `ds_gfs` if you are using GFS gridded analyses.
3. Pay attention to the data and time attributes for the different data plots to make sure you assign the correct variables to each.

### Import Packages

In [ ]:
from datetime import datetime

import metpy.calc as mpcalc
from metpy.plots import declarative
from metpy.units import units
import numpy as np
from siphon.catalog import TDSCatalog
import xarray as xr

### Helper Function

### Get Satellite Data


In [ ]:
date = datetime(2023, 3, 31, 15, 30)

# Create variables for URL generation
region = 'CONUS'
channel = 9
satellite = 'east'

# We want to match something like:
# https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

# Construct the data_url string
data_url = (f'https://thredds.ucar.edu/thredds/catalog/satellite/goes/{satellite}/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/'
            f'{date:%Y%m%d}/catalog.xml')

# Get list of files available for particular day
cat = TDSCatalog(data_url)

# Grab dataset for desired time
dataset = cat.datasets.filter_time_nearest(date, regex=r'_s(?P<strptime>\d{13})', strptime='%Y%j%H%M%S')

# Open most recent file available
ds = dataset.remote_access(use_xarray=True)

# Apply a square root correction for visible imagery only
if channel == 2:
    ds['Sectorized_CMI'].values = np.sqrt(ds['Sectorized_CMI'].values)

# Grab time from file and make into datetime object for plotting and later data access
vtime = ds.time.values.astype('datetime64[ms]').astype('O')

### Get Gridded Data
Choose the closest time to the satellite observations based on the limitations of available gridded data. For example, GFS analysis are only available at 00, 06, 12, and 18 UTC. So for a satellite observation from 1530 UTC, the closest gridded ouput that would be availabe would be 12 UTC.

In [ ]:
gfs_date = datetime(date.year, date.month, date.day, 12)

# Remote access to the dataset from the UCAR site
ds_gfs = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                         f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_{gfs_date:%Y%m%d}_{gfs_date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes
ds_gfs = ds_gfs.sel(lat=slice(70,10), lon=slice(360-150, 360-55)).metpy.parse_cf()

### Plot Imagery

Set attributes for both the contour and satellite image plots, then add both to the same panel.

Note: The extent is set by hand to better correspond to the satellite image area. Trial and error is the best way to come up with an area attribute setting that would best.

In [ ]:
# Declare the data we wish to plot
# Set attributes for plotting contours
cntr = declarative.ContourPlot()
cntr.data = ds_gfs
cntr.field = 'Geopotential_height_isobaric'
cntr.level = 300 * units.hPa
cntr.time = gfs_date
cntr.linecolor = 'cyan'
cntr.contours = list(range(0, 10000, 120))
cntr.clabels = True

img = declarative.ImagePlot()
img.data = ds
img.field = 'Sectorized_CMI'
img.colormap = 'Greys'
img.colorbar = 'horizontal'
img.image_range = (200, 280)

# Plot the data on a map
panel = declarative.MapPanel()
panel.area = [-113, -62, 16, 51]
panel.layers = ['coastline', 'borders', 'states']
panel.left_title = f'GOES East WV Channel {channel:02d}'
panel.right_title = f'Valid Time: {vtime} UTC'
panel.plots = [img, cntr]

# Place the map on a figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]
#pc.save(f'GOES_East_{vtime:%Y%m%d_%H%M}_colorenhanced.png', bbox_inches='tight', dpi=150)
pc.show()